In [1]:
%pip install tensorflow keras numpy pandas matplotlib opencv-python scikit-learn

  Using cached keras-3.9.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached numpy-2.2.3-cp310-cp310-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.me

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [3]:
dataset_path = "/dataset/Train"

In [ ]:
IMG_SIZE = 224  # MobileNetV2 input size
BATCH_SIZE = 32
NUM_CLASSES = 9  # Number of skin cancer classes

# Data Augmentation & Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Load Pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model layers
base_model.trainable = False

# Custom Classification Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

# Create Model
model = Model(inputs=base_model.input, outputs=output)

# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
EPOCHS = 20
model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

# Save Model in Google Drive for Web Deployment
model.save("/skin_cancer_model.h5")

print("Model training complete. Saved as skin_cancer_model.h5 in Google Drive.")

Found 1795 images belonging to 9 classes.
Found 444 images belonging to 9 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1542s 27s/step - accuracy: 0.2152 - loss: 2.1599 - val_accuracy: 0.3806 - val_loss: 1.6519
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 52s 920ms/step - accuracy: 0.3774 - loss: 1.6858 - val_accuracy: 0.4077 - val_loss: 1.6767
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 53s 927ms/step - accuracy: 0.3990 - loss: 1.5999 - val_accuracy: 0.4459 - val_loss: 1.5878
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 54s 944ms/step - accuracy: 0.4622 - loss: 1.4837 - val_accuracy: 0.4527 - val_loss: 1.5336
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 51s 897ms/step - accuracy: 0.4762 - loss: 1.4588 - val_accuracy: 0.4685 - val_loss: 1.5661
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 51s 890ms/step - accuracy: 0.5035 - loss: 1.4124 - val_accuracy: 0.4730 - val_loss: 1.5737
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 50s 882ms/step - accuracy: 0.5048 - loss: 1.3990 - val_accuracy: 0.4730 - val_loss: 1.5684
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 83s 902ms/step - accuracy: 0.4941 - loss: 1.4022 - val_accu

Model training complete. Saved as skin_cancer_model.h5 in Google Drive.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load an image
img_path = "/dataset/Test/nevus/ISIC_0000003.jpg"  # Change this path
img = image.load_img(img_path, target_size=(224, 224))  # Resize to model's input size

# Convert image to array
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize (if your training data is normalized)
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)  # Get the class with highest probability

# Print results
print(f"Predicted Class Index: {predicted_class[0]}")
print(f"Confidence Score: {np.max(predictions):.4f}")

### Try with ISIC__0000013 image from melanoma
### Try with ISIC_0000003 image from nevus

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted Class Index: 4
Confidence Score: 0.9451


In [ ]:
class_indices = train_data.class_indices  # Dictionary mapping class names to indices
print(class_indices)

{'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 3, 'nevus': 4, 'pigmented benign keratosis': 5, 'seborrheic keratosis': 6, 'squamous cell carcinoma': 7, 'vascular lesion': 8}
